In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim
import numpy as np
from torch.utils.data import random_split
from xgboost.sklearn import XGBClassifier


In [201]:
def load_data(filename):
    data = np.loadtxt(filename, delimiter=',', dtype=np.float32)
    x = data[:, 0:-2]
    y = data[:, [-1]] # after 3 months (-2) and 6 months (-1)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [348]:
# target to fix code


class EnhancedFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(EnhancedFNN, self).__init__()
        # 첫 번째 레이어
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ELU(),  # ReLU 활성화 함수 사용
            #nn.BatchNorm1d(hidden_size)  # 배치 정규화 추가
        )
        # 두 번째 레이어: 더 깊은 신경망 구조
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),  # ReLU 활성화 함수 사용
            nn.BatchNo1d(hidden_size)  # 배치 정규화 추가
        )
        # 세 번째 레이어: 출력층
        self.layer3 = nn.Linear(hidden_size, output_size,nn.sigmoid())
        
        # 드롭아웃 추가: 과적합 방지
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)  # 드롭아웃 적용
        x = self.layer3(x)
        #x = self.dropout(x)  # 드롭아웃 적용
        #x = self.layer3(x)
        return x

In [349]:
x_train, y_train = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_training_dataset.csv')
x_test, y_test = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_test_dataset.csv')

In [350]:
y_train

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
      

In [351]:
x_test

tensor([[0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.2000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.3000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 0.7000, 0.6111, 1.0000, 0.1652, 0.4800],
        [0.0042, 0.4179, 0.2535, 0.5000, 0.2500, 0.2727, 0.3986, 0.0000, 0.0000,
         0.2388, 0.6000, 0.5000, 1.0000, 0.1739, 0.7500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 1.0000, 0.7222, 0.0000, 0.1739, 0.0000],
        [0.3208, 0.4467, 0.2676, 0.5000, 0.7500, 0.2727, 0.7489, 0.2812, 0.3559,
         0.1939, 1.0000, 0.7222, 0.0000, 0.3043, 0.7500],
        [0.0084, 0.5389, 0.0000, 1.0000, 0.2500, 0.1818, 0.6514, 0.0000, 0.0735,
         0.3152, 0.4000, 1.0000, 1.0000, 0.2609, 0.0000],
        [0.0168, 0.7176, 0.

In [352]:
x_train.shape

torch.Size([103, 15])

In [353]:


from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test) # test set에는 transform만 사용하기


x_train=torch.Tensor(x_train)
x_test=torch.Tensor(x_test)

In [354]:
model = FNN(input_size=15, hidden_size=300, output_size=1)
criterion = nn.BCELoss()
#optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix
optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix

In [355]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.0443,  0.1121, -0.1695,  ...,  0.0504, -0.2407, -0.2481],
        [-0.2305, -0.1526,  0.1679,  ...,  0.0178,  0.2267,  0.0086],
        [-0.1542,  0.1012, -0.0307,  ..., -0.2081, -0.0755,  0.1271],
        ...,
        [ 0.1137, -0.2488,  0.2277,  ...,  0.0462, -0.0759,  0.0852],
        [-0.2546, -0.1832, -0.1816,  ..., -0.1497,  0.0279, -0.1045],
        [-0.1988,  0.1585, -0.0208,  ..., -0.0108,  0.0903, -0.0710]],
       requires_grad=True)
Parameter containing:
tensor([-0.0880,  0.1352,  0.2014, -0.1323,  0.1730,  0.1185, -0.1670, -0.1356,
         0.1507,  0.1454,  0.0962,  0.0269,  0.1529, -0.0268, -0.0226, -0.1326,
         0.0926,  0.1435,  0.1114,  0.1953,  0.1956,  0.0765, -0.0171, -0.0864,
         0.2455,  0.1498, -0.0181,  0.1596,  0.2567,  0.2576,  0.0408, -0.1533,
         0.1811,  0.0335, -0.1042, -0.2478,  0.0728, -0.1482,  0.1912, -0.0336,
        -0.1705, -0.0614,  0.2541, -0.0027, -0.1797, -0.2074, -0.1438, -0.2012,
        -0.2108

In [356]:
# assuming y is torch.Tensor

In [357]:
num_epochs = 1001

for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')
        

with torch.no_grad(): 
    train_accuracy = (model(x_train).gt(0.5).float() == y_train).float().mean().item()
    test_accuracy = (model(x_test).gt(0.5).float() == y_test).float().mean().item()
    
    

print(f'\nTrain Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
#Train Accuracy: 0.9709 -> start
#Test Accuracy: 0.9000 -> start

Epoch [0/1001], Loss: 0.6904
Epoch [200/1001], Loss: 0.0093
Epoch [400/1001], Loss: 0.0010
Epoch [600/1001], Loss: 0.0013
Epoch [800/1001], Loss: 0.0010
Epoch [1000/1001], Loss: 0.0024

Train Accuracy: 1.0000
Test Accuracy: 0.7500


In [327]:
model(x_test).gt(0.5)

tensor([[False],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False]])

In [177]:
model(x_test).gt(0.5).float()

tensor([[0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.]])

In [178]:
y_test

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]])